<a href="https://colab.research.google.com/github/SmithTheGreat/Romit-Ghosh-MIT-Maker-Portfolio-SRC/blob/main/FPLAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pulp

In [ ]:
import requests
import pandas as pd
import numpy as np
import pulp
import math

In [ ]:
def get_top_5_whitelist():
    data = requests.get("https://fantasy.premierleague.com/api/bootstrap-static/").json()
    elements = pd.DataFrame(data['elements'])
    top5 = elements.sort_values('total_points', ascending=False).head(5)
    return set(top5['web_name'].tolist())

WHITELIST = get_top_5_whitelist()
print("Whitelist (top 5 scorers 24-25):", WHITELIST)

def fetch_fpl_data():
    url = "https://fantasy.premierleague.com/api/bootstrap-static/"
    return requests.get(url).json()

def get_current_season_id():
    url = "https://fantasy.premierleague.com/api/bootstrap-static/"
    data = requests.get(url).json()

    current_season_id = max(event['id'] for event in data['events'])
    return current_season_id

def get_last_season_points(player_id):
    url = f"https://fantasy.premierleague.com/api/element-summary/{player_id}/"
    player_data = requests.get(url).json()
    if 'history_past' in player_data and player_data['history_past']:
        last_season = player_data['history_past'][-1]
        total_points = last_season['total_points']
        season_name = last_season['season_name']
        return season_name, total_points
    return None, 0

def fetch_fixtures():
    url = "https://fantasy.premierleague.com/api/fixtures/"
    return pd.DataFrame(requests.get(url).json())

def get_current_pl_team_ids(data):
    return set(team['id'] for team in data['teams'])

Whitelist (top 5 scorers 24-25): {'Thiago', 'Gabriel', 'Semenyo', 'Haaland', 'Rice'}


In [ ]:
def is_likely_starter(player, current_gw):
    minutes = player.get('minutes', 0)
    selected = float(player.get('selected_by_percent', '0').replace('%', '')) if 'selected_by_percent' in player else 0
    status = player.get('status', 'a')
    name = player.get('web_name', '')

    if status != 'a':
        return False

    possible_minutes_this_season = current_gw * 90 if current_gw > 0 else 90
    has_played_significant_minutes_this_season = minutes >= 0.60 * possible_minutes_this_season


    if has_played_significant_minutes_this_season:
         return True
    if selected >= 8 and status == 'a':
        return True

    return False

In [ ]:
def predict_points_preseason(players_df, fixtures_df, teams_data, lookahead=5):
    team_strength = {}
    for team in teams_data:
        if 'strength' in team:
            team_strength[team['id']] = team['strength']
        elif 'points' in team:
            team_strength[team['id']] = math.floor(5- (3*(team['position']-1)/19))
        else:
            team_strength[team['id']] = 2
    predicted_points_list = []
    current_gw_preseason = 1
    for _, player in players_df.iterrows():
        if not is_likely_starter(player, current_gw_preseason):
            predicted_points_list.append([0] * lookahead)
            continue
        base_points = player.get('total_points', 0)
        games_played = player.get('minutes', 0) / 90 if player.get('minutes', 0) else 1
        base_ppg = base_points / games_played if games_played > 0 else 2.5
        pos = int(player['element_type'])
        def_mult = 1.0
        if pos == 2:
            def_mult = 1.2
        elif pos == 3:
            def_mult = 1.1
        player_fixtures = fixtures_df[
            ((fixtures_df['team_h'] == player['team']) | (fixtures_df['team_a'] == player['team'])) &
            (fixtures_df['event'].notnull())
        ].sort_values('event').head(lookahead)
        gw_points = []
        for gw in range(1, lookahead + 1):
            fixture = player_fixtures[player_fixtures['event'] == gw]
            if fixture.empty:
                gw_points.append(0)
                continue
            fix = fixture.iloc[0]
            if fix['team_h'] == player['team']:
                opp = fix['team_a']
                difficulty = fix['team_h_difficulty']
            else:
                opp = fix['team_h']
                difficulty = fix['team_a_difficulty']
            player_team_strength = team_strength.get(player['team'], 10)
            opp_team_strength = team_strength.get(opp, 10)
            strength_factor = player_team_strength / (opp_team_strength + 1e-5)
            strength_factor = max(0.5, min(1.5, strength_factor))
            ep = base_ppg * def_mult * (6 - difficulty) / 5 * strength_factor
            gw_points.append(ep)
        predicted_points_list.append(gw_points)

    players_df['predicted_points_per_gw'] = predicted_points_list
    players_df['predicted_points'] = players_df['predicted_points_per_gw'].apply(sum)
    return players_df

In [ ]:
def factor_weight(current_gw):
    w_form=min(0.4,(current_gw-1)/4*0.4)if current_gw>=1 else 0.0
    w_last=max(0.0,0.6*(1-(current_gw-1)/37))
    w_current=min(0.2,0.2*(current_gw-1)/37)
    w_fixture=0.4
    total=w_form+w_last+w_current+w_fixture
    w_form/=total
    w_last/=total
    w_current/=total
    w_fixture/=total
    return w_last,w_current,w_form,w_fixture

def last5_form(points_list,current_gw):
    if current_gw<=1 or not points_list: return 0.0
    start=max(0,current_gw-5)
    window=points_list[start:current_gw-1]
    if not window: return 0.0
    weights=np.linspace(0.5,1.0,len(window))
    return float(np.average(window,weights=weights))

def get_last_season_ppg(player):
    if 'history_past'in player and player['history_past']:
        minutes=player['history_past'][-1]['minutes']
        points=player['history_past'][-1]['total_points']
        return points/(minutes/90 if minutes>0 else 1)
    return 0

def get_current_season_ppg(player, current_gw):
    if current_gw <= 1:
        return 0
    if 'history' in player and player['history']:
        minutes = sum(h['minutes'] for h in player['history'])
        points = sum(h['total_points'] for h in player['history'])
        return points / (minutes / 90) if minutes > 0 else points
    return 0

def predict_points_future_custom(player, current_gw, fixtures_df, teams_data, lookahead=5):
    _,last_season_points=get_last_season_points(player['id'])
    ppg_last=last_season_points/38 if last_season_points>0 else 0
    ppg_current=get_current_season_ppg(player,current_gw)
    points_list=[h['total_points'] for h in player.get('history',[])]
    form_ppg=last5_form(points_list,current_gw)

    if not is_likely_starter(player, current_gw):
        return [0]*lookahead

    w_last,w_current,w_form,w_fixture = factor_weight(current_gw)

    base_ppg=w_last*ppg_last+w_current*ppg_current+w_form*form_ppg
    base_ppg=max(base_ppg,0.5)

    team_strength={team['id']:team.get('strength',10) for team in teams_data}
    predicted_points_list=[]
    for gw_offset in range(lookahead):
        gw=current_gw+gw_offset
        player_fixtures=fixtures_df[((fixtures_df['team_h']==player['team'])|(fixtures_df['team_a']==player['team']))&(fixtures_df['event']==gw)]
        if player_fixtures.empty:
            predicted_points_list.append(base_ppg)
            continue

        fix=player_fixtures.iloc[0]
        if fix['team_h']==player['team']:
            opp=fix['team_a']
            difficulty=fix['team_h_difficulty']
        else:
            opp=fix['team_h']
            difficulty=fix['team_a_difficulty']

        team_str=team_strength.get(player['team'],10)
        opp_str=team_strength.get(opp,10)
        strength_factor=np.clip(team_str/(opp_str+1e-5),0.5,1.5)
        fixture_factor=(6-difficulty)/5
        ep=(1-w_fixture)*base_ppg+w_fixture*(base_ppg*fixture_factor*strength_factor)
        predicted_points_list.append(ep)

    return [p * 2 for p in predicted_points_list]


def suggest_transfers(current_squad, all_players, current_gw, fixtures_df, teams_data, max_transfers, budget, squad_cost, transfer_cost=4):
    predicted_points = {p['id']: sum(predict_points_future_custom(p, current_gw, fixtures_df, teams_data, lookahead=5)) for p in all_players}
    current_squad_cost = squad_cost
    swaps = []
    for player_out in current_squad:
        for player_in in all_players:
            if player_in['element_type'] != player_out['element_type']:
                continue
            if player_in['id'] in [p['id'] for p in current_squad]:
                continue
            if predicted_points.get(player_in['id'], 0) <= 0:
                continue
            new_squad_cost = current_squad_cost - player_out['now_cost'] + player_in['now_cost']
            if new_squad_cost > budget:
                continue


            gain_before_cost = predicted_points[player_in['id']] - predicted_points[player_out['id']]
            gain_after_cost = gain_before_cost - transfer_cost

            if gain_after_cost > 0:
                 swaps.append({'out': player_out, 'in': player_in, 'gain_before_cost': gain_before_cost, 'gain': gain_after_cost})

    swaps.sort(key=lambda x: x['gain'], reverse=True)
    selected_swaps = []
    incoming_players_used = set()
    for swap in swaps:
        if swap['in']['id'] not in incoming_players_used and len(selected_swaps) < max_transfers:
            selected_swaps.append(swap)
            incoming_players_used.add(swap['in']['id'])

    return selected_swaps


def select_captains(starting_xi,current_gw,fixtures_df,teams_data):
    predicted={p['id']: predict_points_future_custom(p,current_gw,fixtures_df,teams_data, lookahead=1)[0]*5 for p in starting_xi}
    sorted_players=sorted(starting_xi,key=lambda x:predicted[x['id']],reverse=True)
    return sorted_players[0],sorted_players[1]

def chip_recommendation(squad,current_gw,fixtures_df,teams_data):
    bench_points=[sum(predict_points_future_custom(p,current_gw,fixtures_df,teams_data, lookahead=5)) for p in squad[11:]]
    bench_boost_suggested=np.mean(bench_points)>5
    captain_points=sum(predict_points_future_custom(squad[0],current_gw,fixtures_df,teams_data, lookahead=5))
    triple_captain_suggested=captain_points>8
    squad_points=sum([sum(predict_points_future_custom(p,current_gw,fixtures_df,teams_data, lookahead=5)) for p in squad])
    free_hit_suggested=squad_points<50
    return{'bench_boost':bench_boost_suggested,'triple_captain':triple_captain_suggested,'free_hit':free_hit_suggested}

In [ ]:
def optimize_squad(players_df, budget=1000):
    players_df = players_df.reset_index(drop=True)
    prob = pulp.LpProblem("FPL_Squad_Selection", pulp.LpMaximize)
    x = [pulp.LpVariable(f"x{i}", cat="Binary") for i in range(len(players_df))]
    prob += pulp.lpSum([x[i] * players_df.loc[i, 'predicted_points'] for i in range(len(players_df))])
    prob += pulp.lpSum(x) == 15
    prob += pulp.lpSum([x[i] for i in range(len(players_df)) if players_df.loc[i, 'element_type'] == 1]) == 2
    prob += pulp.lpSum([x[i] for i in range(len(players_df)) if players_df.loc[i, 'element_type'] == 2]) == 5
    prob += pulp.lpSum([x[i] for i in range(len(players_df)) if players_df.loc[i, 'element_type'] == 3]) == 5
    prob += pulp.lpSum([x[i] for i in range(len(players_df)) if players_df.loc[i, 'element_type'] == 4]) == 3
    teams = players_df['team'].unique()
    for team in teams:
        prob += pulp.lpSum([x[i] for i in range(len(players_df)) if players_df.loc[i, 'team'] == team]) <= 3
    prob += pulp.lpSum([x[i] * players_df.loc[i, 'now_cost'] for i in range(len(players_df))]) <= budget
    solver = pulp.PULP_CBC_CMD(msg=0)
    prob.solve(solver)
    selected_indices = [i for i in range(len(players_df)) if pulp.value(x[i]) == 1]
    squad = players_df.loc[selected_indices]
    best_players = {}
    for pos, pos_name in zip([1, 2, 3, 4], ["GK", "DEF", "MID", "FWD"]):
        top5 = players_df[players_df['element_type'] == pos].nlargest(5, 'predicted_points')
        best_players[pos_name] = top5[['web_name', 'predicted_points']].reset_index(drop=True)

    return squad, best_players

In [ ]:
def build_starting_lineups(squad, fixtures_df, lookahead=5):
    rotations = {}
    per_gw_points = {}
    for _, player in squad.iterrows():
        per_gw_points[player['id']] = player['predicted_points_per_gw']
    for gw in range(lookahead):
        squad['gw_predicted_points'] = squad['id'].apply(lambda pid: per_gw_points[pid][gw])
        squad_sorted = squad.sort_values(by='gw_predicted_points', ascending=False)
        starting_xi = squad_sorted.head(11).copy()
        count_gk = sum(starting_xi['element_type'] == 1)
        count_def = sum(starting_xi['element_type'] == 2)
        count_mid = sum(starting_xi['element_type'] == 3)
        count_fwd = sum(starting_xi['element_type'] == 4)

        def swap_in_player(pos_needed, exclude_pos):
            nonlocal starting_xi, squad_sorted
            candidates_in = squad_sorted[(squad_sorted['element_type'] == pos_needed) & (~squad_sorted.index.isin(starting_xi.index))]
            if candidates_in.empty:
                return False
            player_in = candidates_in.iloc[0]

            removable = starting_xi[~starting_xi['element_type'].isin(exclude_pos)].sort_values(by='gw_predicted_points')
            if removable.empty:
                return False
            player_out = removable.iloc[0]

            starting_xi = starting_xi.drop(player_out.name)
            starting_xi = pd.concat([starting_xi, player_in.to_frame().T])
            return True
        if count_gk == 0:
            swap_in_player(1, exclude_pos=[])
        elif count_gk > 1:
            gks = starting_xi[starting_xi['element_type'] == 1].sort_values(by='gw_predicted_points')
            to_drop = gks.iloc[:-1]
            starting_xi = starting_xi.drop(to_drop.index)
        count_gk = sum(starting_xi['element_type'] == 1)
        count_def = sum(starting_xi['element_type'] == 2)
        count_mid = sum(starting_xi['element_type'] == 3)
        count_fwd = sum(starting_xi['element_type'] == 4)
        while count_def < 3:
            if not swap_in_player(2, exclude_pos=[1, 2]):
                break
            count_def += 1
            count_mid = sum(starting_xi['element_type'] == 3)
            count_fwd = sum(starting_xi['element_type'] == 4)
        while count_mid < 3:
            if not swap_in_player(3, exclude_pos=[1, 2, 3]):
                break
            count_mid += 1
            count_def = sum(starting_xi['element_type'] == 2)
            count_fwd = sum(starting_xi['element_type'] == 4)
        while count_fwd < 2:
            if not swap_in_player(4, exclude_pos=[1, 2, 3, 4]):
                break
            count_fwd += 1
            count_def = sum(starting_xi['element_type'] == 2)
            count_mid = sum(starting_xi['element_type'] == 3)
        if len(starting_xi) < 11:
            needed = 11 - len(starting_xi)
            extras = squad_sorted[~squad_sorted.index.isin(starting_xi.index)].head(needed)
            starting_xi = pd.concat([starting_xi, extras])
        if len(starting_xi) > 11:
            starting_xi = starting_xi.head(11)
        subs = squad_sorted[~squad_sorted.index.isin(starting_xi.index)].head(4)
        rotations[gw + 1] = {
            'starting_xi': starting_xi.sort_values(by='gw_predicted_points', ascending=False),
            'subs': subs
        }
    return rotations

In [ ]:
def print_rotation_plan(rotations, fixtures_df):
    pos_map = {1: 'GK', 2: 'DEF', 3: 'MID', 4: 'FWD'}
    pos_order = [4, 3, 2, 1]
    rotation_output = ""
    for gw, lineup in rotations.items():
        rotation_output += f"Gameweek {gw}:\n"
        starters = lineup['starting_xi'].copy()
        if len(starters) != 11:
            rotation_output += f"Warning: Starting XI size is {len(starters)} players, expected 11.\n"
        fixtures_gw = fixtures_df[fixtures_df['event'] == gw]

        def get_fixture_info(player):
            fix = fixtures_gw[(fixtures_gw['team_h'] == player['team']) | (fixtures_gw['team_a'] == player['team'])]
            if fix.empty:
                return ("No Fixture", None)
            fix = fix.iloc[0]
            opp = fix['team_a'] if fix['team_h'] == player['team'] else fix['team_h']
            diff = fix['team_h_difficulty'] if fix['team_h'] == player['team'] else fix['team_a_difficulty']
            return (opp, diff)
        starters['next_fixture'], starters['difficulty'] = zip(*starters.apply(get_fixture_info, axis=1))
        starters['pos_name'] = starters.apply(lambda r: f"{pos_map[r['element_type']]} {r['web_name']}", axis=1)
        starters['pos_order'] = starters['element_type'].apply(lambda x: pos_order.index(x))
        starters = starters.sort_values(by='pos_order')
        rotation_output += " Starting XI:\n"
        rotation_output += starters[['pos_name', 'team', 'gw_predicted_points', 'next_fixture', 'difficulty']].to_string(index=False) + "\n"
        subs = lineup['subs'].copy()
        if len(subs) != 4:
            rotation_output += f"Warning: Subs size is {len(subs)} players, expected 4.\n"
        subs['next_fixture'], subs['difficulty'] = zip(*subs.apply(get_fixture_info, axis=1))
        subs['pos_name'] = subs.apply(lambda r: f"{pos_map[r['element_type']]} {r['web_name']}", axis=1)
        subs['pos_order'] = subs['element_type'].apply(lambda x: pos_order.index(x))
        subs = subs.sort_values(by='pos_order')
        rotation_output += " Subs:\n"
        rotation_output += subs[['pos_name', 'team', 'gw_predicted_points', 'next_fixture', 'difficulty']].to_string(index=False) + "\n"
        rotation_output += "\n"
    return rotation_output

In [ ]:
lookahead= 5
data=fetch_fpl_data()
current_gw = 21
elements=pd.DataFrame(data['elements'])
fixtures=fetch_fixtures()
pl_team_ids=get_current_pl_team_ids(data)

players=elements[elements['team'].isin(pl_team_ids)][['id','web_name','team','now_cost','element_type','minutes','total_points','selected_by_percent','status']]
player_list=players.to_dict('records')
for p in player_list:
    predicted_list =predict_points_future_custom(p,current_gw=current_gw,fixtures_df=fixtures, teams_data=data['teams'], lookahead = lookahead)
    p['predicted_points_per_gw'] = predicted_list
    p['predicted_points'] = sum(predicted_list)
players_df = pd.DataFrame(player_list)
squad,best_players=optimize_squad(players_df,budget=1000)
rotations=build_starting_lineups(squad,fixtures,lookahead)
final_squad_text=squad[['web_name','element_type','team','now_cost','predicted_points']].to_string(index=False)
rotation_text=print_rotation_plan(rotations,fixtures)
captain,vice_captain=select_captains(starting_xi=squad.head(11).to_dict('records'),current_gw=current_gw,fixtures_df=fixtures,teams_data=data['teams'])
captain_text=f"Captain: {captain['web_name']}, Vice-Captain: {vice_captain['web_name']}"
chips=chip_recommendation(squad=squad.to_dict('records'),current_gw=current_gw,fixtures_df=fixtures,teams_data=data['teams'])
chip_text="\n".join([f"{chip}: {'Yes' if suggested else 'No'}" for chip,suggested in chips.items()])
best_players_text="\n".join([f"\n{pos}:\n{df.to_string(index=False)}" for pos,df in best_players.items()])

In [ ]:
output = f"""
Final squad (15 players):
{final_squad_text}

Rotation plan for next {lookahead} GWs:
{rotation_text}

{captain_text}

Chip Recommendations:
{chip_text}

Top 5 per position:
{best_players_text}
"""
print(output)


Final squad (15 players):
  web_name  element_type  team  now_cost  predicted_points
    Saliba             2     1        59          7.643795
       Eze             3     1        74          8.702167
   Watkins             4     2        87          9.703984
   Semenyo             3     4        76          8.676708
   Collins             2     5        49          6.836227
    Palmer             3     7       104         11.696568
  Pickford             1     9        56          8.635784
      Raúl             4    10        62          7.912798
    Virgil             2    12        59          7.555348
Szoboszlai             3    12        66          7.555348
     Cunha             3    14        82          9.153915
      Sels             1    16        46          7.850643
Milenković             2    16        51          7.588955
   Murillo             2    16        52          6.803890
     Bowen             4    19        77         10.708637

Rotation plan for next 5 GWs

In [ ]:
print(best_players_text)


GK:
 web_name  predicted_points
 Pickford          8.635784
     Raya          8.349377
     Sels          7.850643
Henderson          7.155035
     Leno          6.190284

DEF:
  web_name  predicted_points
    Saliba          7.643795
Milenković          7.588955
    Virgil          7.555348
    Kerkez          7.079836
   Gabriel          6.879416

MID:
web_name  predicted_points
  Palmer         11.696568
   Cunha          9.153915
     Eze          8.702167
 Semenyo          8.676708
  Rogers          8.399685

FWD:
web_name  predicted_points
   Bowen         10.708637
 Haaland         10.242712
 Watkins          9.703984
  Mateta          7.950039
    Raúl          7.912798


In [ ]:
print(factor_weight(21))

(0.2328767123287671, 0.09132420091324202, 0.33789954337899547, 0.33789954337899547)


In [ ]:
def get_player_by_name(player_name, all_players_df):
    return all_players_df[all_players_df['web_name'].str.lower() == player_name.lower()].iloc[0]

current_squad_names = []

print("Enter the web names for your squad by position.")

gk_names_input = input("Enter the web names of your 2 Goalkeepers, separated by commas: ")
current_squad_names.extend([name.strip() for name in gk_names_input.split(',')])

def_names_input = input("Enter the web names of your 5 Defenders, separated by commas: ")
current_squad_names.extend([name.strip() for name in def_names_input.split(',')])

mid_names_input = input("Enter the web names of your 5 Midfielders, separated by commas: ")
current_squad_names.extend([name.strip() for name in mid_names_input.split(',')])

fwd_names_input = input("Enter the web names of your 3 Forwards, separated by commas: ")
current_squad_names.extend([name.strip() for name in fwd_names_input.split(',')])

print("Processed names:", current_squad_names)

try:
    current_squad_list = [get_player_by_name(name, players_df) for name in current_squad_names]
    current_squad_df = pd.DataFrame(current_squad_list)

    current_gw = 12

    current_squad_cost = current_squad_df['now_cost'].sum()

    squad_budget = 1000


    print(f"\nCurrent squad cost: {current_squad_cost:.2f}")
    transfer_budget_limit = squad_budget


    print("\nPredicted points for your current squad over the next 5 GWs:")
    for _, player in current_squad_df.iterrows():
        predicted_points_gw = predict_points_future_custom(
            player.to_dict(),
            current_gw=current_gw,
            fixtures_df=fixtures,
            teams_data=data['teams'],
            lookahead=5
        )
        total_predicted_points = sum(predicted_points_gw)
        print(f"{player['web_name']}: {total_predicted_points:.2f}")


    transfer_suggestions = suggest_transfers(
        current_squad=current_squad_df.to_dict('records'),
        all_players=players_df.to_dict('records'),
        current_gw=current_gw,
        fixtures_df=fixtures,
        teams_data=data['teams'],
        max_transfers=5,
        budget=transfer_budget_limit,
        squad_cost=current_squad_cost,
        transfer_cost=4
    )

    print("\nSuggested transfers (max 5, -4 each):")
    if transfer_suggestions:
        for t in transfer_suggestions:
            print(f"Out: {t['out']['web_name']} | In: {t['in']['web_name']} | Expected Gain: {t['gain_before_cost']:.2f} (before cost), {t['gain']:.2f} (after cost)")
    else:
        print("No beneficial transfers suggested within budget.")

except IndexError:
    print("Error: Could not find one or more players. Please check the names.")
except Exception as e:
    print(f"An error occurred: {e}")

Enter the web names for your squad by position.
Enter the web names of your 2 Goalkeepers, separated by commas: Dúbravka,Sánchez
Enter the web names of your 5 Defenders, separated by commas: Collins,Guéhi,Gabriel,Anderson,Chalobah
Enter the web names of your 5 Midfielders, separated by commas: Garner,Foden,Rice,Bruno G.,Cunha
Enter the web names of your 3 Forwards, separated by commas: Thiago,Haaland,Calvert-Lewin
Processed names: ['Dúbravka', 'Sánchez', 'Collins', 'Guéhi', 'Gabriel', 'Anderson', 'Chalobah', 'Garner', 'Foden', 'Rice', 'Bruno G.', 'Cunha', 'Thiago', 'Haaland', 'Calvert-Lewin']

Current squad cost: 1009.00

Predicted points for your current squad over the next 5 GWs:
Dúbravka: 4.38
Sánchez: 0.00
Collins: 9.57
Guéhi: 8.85
Gabriel: 10.01
Anderson: 8.05
Chalobah: 6.97
Garner: 4.38
Foden: 8.68
Rice: 10.86
Bruno G.: 10.16
Cunha: 14.07
Thiago: 4.35
Haaland: 14.96
Calvert-Lewin: 4.57

Suggested transfers (max 5, -4 each):
Out: Foden | In: Trossard | Expected Gain: 4.32 (before 

In [ ]:
def find_best_player_by_position_and_budget(players_df, position_name, max_budget_million):
    position_map = {'GK': 1, 'DEF': 2, 'MID': 3, 'FWD': 4}
    position_id = position_map.get(position_name.upper())
    if position_id is None:
        print(f"Error: Invalid position entered. Please use GK, DEF, MID, or FWD.")
        return None

    max_budget_cost = int(max_budget_million * 10)
    filtered_players = players_df[
        (players_df['element_type'] == position_id) &
        (players_df['now_cost'] <= max_budget_cost)
    ]
    best_player = filtered_players.sort_values(by='predicted_points', ascending=False)
    if not best_player.empty:
        top_player = best_player.iloc[0]
        print(f"\nBest {position_name.upper()} for {max_budget_million:.1f} million or less:")
        print(f"Name: {top_player['web_name']}")
        print(f"Predicted Points (next 5 GWs): {top_player['predicted_points']:.2f}")
        print(f"Cost: {top_player['now_cost'] / 10.0:.1f} million")
        return top_player
    else:
        print(f"\nNo {position_name.upper()} players found for {max_budget_million:.1f} million or less with positive predicted points.")
        return None
position_input = input("Enter the desired position (GK, DEF, MID, FWD): ")
try:
    budget_input_million = float(input("Enter the maximum budget in millions (e.g., 6.4): "))
    find_best_player_by_position_and_budget(players_df, position_input, budget_input_million)
except ValueError:
    print("Invalid budget entered. Please enter a number (e.g., 6.4).")
except Exception as e:
    print(f"An error occurred: {e}")

Enter the desired position (GK, DEF, MID, FWD): GK
Enter the maximum budget in millions (e.g., 6.4): 7.4

Best GK for 7.4 million or less:
Name: Pickford
Predicted Points (next 5 GWs): 8.64
Cost: 5.6 million
